In [4]:


# Example of creating, removing, and viewing my-python-env containers
#   and printing system information
# See container/Dockerfile for definition of my-python-env

import requests
public_ip = requests.get('https://ipconfig.io/ip').text.strip()
print(f"Local IP: {public_ip}")


Local IP: 150.109.4.158


In [14]:
import docker
from IPython.display import display

# shortcut of docker Container class
Container = docker.models.containers.Container
display(dir(Container))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'attach',
 'attach_socket',
 'commit',
 'diff',
 'exec_run',
 'export',
 'get_archive',
 'health',
 'id',
 'id_attribute',
 'image',
 'kill',
 'labels',
 'logs',
 'name',
 'pause',
 'ports',
 'put_archive',
 'reload',
 'remove',
 'rename',
 'resize',
 'restart',
 'short_id',
 'start',
 'stats',
 'status',
 'stop',
 'top',
 'unpause',
 'update',
 'wait']

In [5]:


import platform
print(f"CPU name: {platform.processor()}")

import psutil
# cpu
cpu_cores = psutil.cpu_count(logical=False)
logical_cores = psutil.cpu_count(logical=True)
print(f"Physical CPU cores: {cpu_cores}")
print(f"Logical CPU cores: {logical_cores}")

# memory
memory_info = psutil.virtual_memory()
total_memory = memory_info.total
available_memory = memory_info.available
print(f"Total memory: {total_memory} bytes")
print(f"Available memory: {available_memory} bytes")

# disk
disk_info = psutil.disk_usage('/')
total_disk_space = disk_info.total
used_disk_space = disk_info.used
free_disk_space = disk_info.free
print(f"Total disk space: {total_disk_space} bytes")
print(f"Used disk space: {used_disk_space} bytes")
print(f"Free disk space: {free_disk_space} bytes")


CPU name: x86_64
Physical CPU cores: 2
Logical CPU cores: 2
Total memory: 3565187072 bytes
Available memory: 1391464448 bytes
Total disk space: 63290032128 bytes
Used disk space: 11983147008 bytes
Free disk space: 48606138368 bytes


In [6]:


import threading
import time
import random
import docker
daemon = docker.from_env()

for container in daemon.containers.list(all=True):
    container.remove(force=True)

def new_container():
    password = str(random.randint(100000, 999999))
    container = daemon.containers.run(
        "my-python-env",
        detach=True,
        ports={
            # exposes container's ssh, jupyter, and http ports to the host
            # 0 means we let docker choose a random available port on the host
            22: 0,
            8888: 0,
            80: 0,
        },
        cpuset_cpus="0,1", # cpus number 0 to 1
        mem_limit="1g", # 1 gigabyte
        device_requests=[
            # docker.types.DeviceRequest(device_ids=["0,1"], capabilities=[['gpu']])
        ],
        labels=["my-python-env"],
        environment={
            "PASSWORD": password,
        },
    )
    return container

def log_container(container):
    print(container.short_id)

    # container object only stores the id
    # container.attrs will only be updated after reload()
    # we will get empty dict if we don't reload
    container.reload()
    ports = container.attrs["NetworkSettings"]["Ports"]
    ssh_port = ports["22/tcp"][0]["HostPort"]
    jupyter_port = ports["8888/tcp"][0]["HostPort"]
    print("mapped ports", ssh_port, jupyter_port)
    print("ssh command:")
    print(f"    ssh -p {ssh_port} root@{public_ip}")

    password = container.attrs["Config"]["Env"][0].split("=")[1]
    print(f"    password", password)

    # we also need to get jupyter login token
    # the CMD in dockerfile contains "jupyter lab ..."
    # which will print something like:
    # Or copy and paste one of these URLs:
    #     http://878f1a9b4dfa:8888/lab?token=d336fa63c03f064ff15ce7b269cab95b2095786cf9ab2ba3
    #  or http://127.0.0.1:8888/lab?token=d336fa63c03f064ff15ce7b269cab95b2095786cf9ab2ba3
    # we can parse these and try to get the token
    # if the container is still starting up, we may not get the token
    print("wait for logs:")
    time.sleep(2)
    logs = container.logs().decode("utf-8")
    line = [line for line in logs.split("\n") if "?token=" in line]
    if len(line) == 0:
        print("no token found. Maybe you need to wait a bit longer for jupyter to log the token.")
    else:
        token = line[0].split("?token=")[1]
        print("token", token)
        print("jupyter url:")
        print(f"    http://{public_ip}:{jupyter_port}/lab?token={token}")

    print()
    print('Specs:')
    print("cpus", container.attrs["HostConfig"]["CpusetCpus"])
    print("memory", container.attrs["HostConfig"]["MemorySwap"])
    print("gpus", container.attrs["HostConfig"]["DeviceRequests"])

    print()
    print("complete container info json:")
    display(container.attrs)

log_container(new_container())


ba7ce70b2e03
mapped ports 32989 32987
ssh command:
    ssh -p 32989 root@150.109.4.158
    password 459168
wait for logs:
no token found. Maybe you need to wait a bit longer for jupyter to log the token.

Specs:
cpus 0,1
memory 2147483648
gpus []

complete container info json:


{'Id': 'ba7ce70b2e03678099d40e2610be11b45a09a837f904054cba8a8c196c79440f',
 'Created': '2024-01-19T14:10:26.206380652Z',
 'Path': '/bin/sh',
 'Args': ['-c',
  'echo "root:"${PASSWORD} | chpasswd &&     /usr/sbin/sshd -D &     curl ipconfig.io &&     jupyter lab --allow-root --ip=0.0.0.0 --port=8888 &     echo done &&     sleep infinity'],
 'State': {'Status': 'running',
  'Running': True,
  'Paused': False,
  'Restarting': False,
  'OOMKilled': False,
  'Dead': False,
  'Pid': 428690,
  'ExitCode': 0,
  'Error': '',
  'StartedAt': '2024-01-19T14:10:26.746264986Z',
  'FinishedAt': '0001-01-01T00:00:00Z'},
 'Image': 'sha256:5c86cad0b43ae0ceaba2e38304898e53a39e0b3b24ca69fbdd5ce6879cfc92dd',
 'ResolvConfPath': '/var/lib/docker/containers/ba7ce70b2e03678099d40e2610be11b45a09a837f904054cba8a8c196c79440f/resolv.conf',
 'HostnamePath': '/var/lib/docker/containers/ba7ce70b2e03678099d40e2610be11b45a09a837f904054cba8a8c196c79440f/hostname',
 'HostsPath': '/var/lib/docker/containers/ba7ce70b2e0367

In [7]:


def log_container_json(container):
    # summarize container info above into a json
    global daemon, public_ip
    short_id = container.short_id
    container.reload()
    status = container.status
    ports = container.attrs["NetworkSettings"]["Ports"]
    ssh_port = ports["22/tcp"][0]["HostPort"]
    jupyter_port = ports["8888/tcp"][0]["HostPort"]

    ssh_command = f"ssh -p {ssh_port} root@{public_ip}"
    password = container.attrs["Config"]["Env"][0].split("=")[1]

    logs = container.logs().decode("utf-8")
    line = [line for line in logs.split("\n") if "?token=" in line]
    if len(line) == 0:
        jupyter_token = ""
        jupyter_command = ""
    else:
        token = line[0].split("?token=")[1]
        jupyter_token = token
        jupyter_command = f"http://{public_ip}:{jupyter_port}/lab?token={token}"
    
    cpus = container.attrs["HostConfig"]["CpusetCpus"]
    memory = container.attrs["HostConfig"]["MemorySwap"]
    gpus = container.attrs["HostConfig"]["DeviceRequests"]

    keys = [
        "short_id",
        "status",
        "ssh_port",
        "jupyter_port",
        "ssh_command",
        "password",
        "jupyter_token",
        "jupyter_command",
        "cpus",
        "memory",
        "gpus",
    ]
    vars = locals()
    return {key: vars[key] for key in keys} 


container = new_container()
display(log_container_json(container))
print("now wait for jupyter to log the token")
time.sleep(2)
display(log_container_json(container))

for container in daemon.containers.list(all=True):
    container.remove(force=True)


{'short_id': '7a0167637689',
 'status': 'running',
 'ssh_port': '32992',
 'jupyter_port': '32990',
 'ssh_command': 'ssh -p 32992 root@150.109.4.158',
 'password': '300884',
 'jupyter_token': '',
 'jupyter_command': '',
 'cpus': '0,1',
 'memory': 2147483648,
 'gpus': []}

now wait for jupyter to log the token


{'short_id': '7a0167637689',
 'status': 'running',
 'ssh_port': '32992',
 'jupyter_port': '32990',
 'ssh_command': 'ssh -p 32992 root@150.109.4.158',
 'password': '300884',
 'jupyter_token': 'ee2bb93d917a6866a1c09c16179a890748999d5e73cc2cbf',
 'jupyter_command': 'http://150.109.4.158:32990/lab?token=ee2bb93d917a6866a1c09c16179a890748999d5e73cc2cbf',
 'cpus': '0,1',
 'memory': 2147483648,
 'gpus': []}